In [15]:
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import os
import pandas as pd

geom_file="/Users/afox/Jedi/Jedi_experiments/input/wrf_hydro_nwm_files/geometry_cfs_long_range_ens.nc"
ncfile = netCDF4.Dataset(geom_file, mode='r')

# Get metadata from the file
npx = ncfile.dimensions["west_east"].size
npy = ncfile.dimensions["south_north"].size
npz = 1

lons = ncfile.variables["XLONG"][:]
lats = ncfile.variables["XLAT"][:]

# Print field dimensions
print(" Grid dimensions", npx, 'x', npy, 'x', npz)

# Zero out array to fill with field
hgt = np.zeros((npy, npx))

hgt[:,:] = ncfile.variables["HGT"][:,:]

 Grid dimensions 4608 x 3840 x 1


In [16]:
obs_file = '/Users/afox/Jedi/Jedi_experiments/OWP_obs/wdb0_obs_swe_snwd_target_2020030100_p24h_m24h_max_sep_6h_test02.csv'
df = pd.read_csv(obs_file)
df

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,0.152400,0.04,0,NaN,NaN,44.050000,-96.600000,2020-02-29T13:00:00Z,snow_depth_m
1,99999,XXXX1,0.000000,0.04,0,NaN,NaN,48.350000,-100.400000,2020-03-01T14:00:00Z,snow_depth_m
2,99999,XXXX1,0.175260,0.04,0,NaN,NaN,41.275000,-122.720000,2020-03-01T00:00:00Z,snow_depth_m
3,99999,XXXX1,1.046480,0.04,0,NaN,NaN,41.350000,-122.245000,2020-02-29T12:00:00Z,snow_depth_m
4,99999,XXXX1,1.102614,0.04,0,NaN,NaN,38.450000,-120.035000,2020-03-01T00:00:00Z,snow_depth_m
...,...,...,...,...,...,...,...,...,...,...,...
3769,99999,XXXX1,56.387998,10.00,0,NaN,NaN,40.714203,-111.301292,2020-03-01T14:00:00Z,snow_water_equivalent_mm
3770,99999,XXXX1,129.539996,10.00,0,NaN,NaN,42.742200,-78.208500,2020-02-29T12:00:00Z,snow_water_equivalent_mm
3771,99999,XXXX1,0.000000,10.00,0,NaN,NaN,32.770530,-81.545560,2020-02-29T12:00:00Z,snow_water_equivalent_mm
3772,99999,XXXX1,332.740009,10.00,0,NaN,NaN,62.950000,-155.610000,2020-03-01T00:00:00Z,snow_water_equivalent_mm


In [17]:
latitude = df['latitude']
longitude = df['longitude']

nobs = np.shape(longitude)
nobs = nobs[0]

In [18]:
lon_req = longitude
lat_req = latitude

array1 = np.asarray(lons)
array2 = np.asarray(lats)

height_out = np.zeros(nobs)

for i in range(nobs):
    lons_diff = (array1 - (lon_req[i]))**2
    lats_diff = (array2 - (lat_req[i]))**2
    comb_diff = lons_diff+lats_diff
    idx = comb_diff.argmin()
    height_out[i] = hgt.flatten()[idx]

In [22]:
df['dem_elev_m']=height_out
df['rec_elev_m']=height_out

,StnObjID,StnID,ObsValue,ObsError,PreQC,dem_elev_m,rec_elev_m,latitude,longitude,datetime,variable_name
0,99999,XXXX1,0.152400,0.04,0,472.513794,472.513794,44.050000,-96.600000,2020-02-29T13:00:00Z,snow_depth_m
1,99999,XXXX1,0.000000,0.04,0,452.590454,452.590454,48.350000,-100.400000,2020-03-01T14:00:00Z,snow_depth_m
2,99999,XXXX1,0.175260,0.04,0,1711.306152,1711.306152,41.275000,-122.720000,2020-03-01T00:00:00Z,snow_depth_m
3,99999,XXXX1,1.046480,0.04,0,2052.353516,2052.353516,41.350000,-122.245000,2020-02-29T12:00:00Z,snow_depth_m
4,99999,XXXX1,1.102614,0.04,0,2198.099121,2198.099121,38.450000,-120.035000,2020-03-01T00:00:00Z,snow_depth_m
...,...,...,...,...,...,...,...,...,...,...,...
3769,99999,XXXX1,56.387998,10.00,0,1981.135620,1981.135620,40.714203,-111.301292,2020-03-01T14:00:00Z,snow_water_equivalent_mm
3770,99999,XXXX1,129.539996,10.00,0,540.465637,540.465637,42.742200,-78.208500,2020-02-29T12:00:00Z,snow_water_equivalent_mm
3771,99999,XXXX1,0.000000,10.00,0,28.768526,28.768526,32.770530,-81.545560,2020-02-29T12:00:00Z,snow_water_equivalent_mm
3772,99999,XXXX1,332.740009,10.00,0,0.000000,0.000000,62.950000,-155.610000,2020-03-01T00:00:00Z,snow_water_equivalent_mm


In [23]:
df.to_csv (r'/Users/afox/Jedi/Jedi_experiments/OWP_obs/wdb0_obs_swe_snwd_target_2020030100_p24h_m24h_max_sep_6h_test02_elevation_filled.csv', index = False, header=True)